### Student Information
Name:

Student ID: 108048110

GitHub ID: MeganKuo0704

Kaggle name:

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 15th 11:59 pm, Tuesday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 18th 11:59 pm, Friday)__. 

In [1]:
### Begin Assignment Here
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder

In [2]:
id = pd.read_csv('/home/project_student/megan/new_env/DM/Lab2/dataset/data_identification.csv')
emo = pd.read_csv('/home/project_student/megan/new_env/DM/Lab2/dataset/emotion.csv')
tweets = pd.read_json('/home/project_student/megan/new_env/DM/Lab2/dataset/tweets_DM.json', lines=True)

In [3]:
train_id = id[id.identification=='train']
train_emo = emo[emo.tweet_id.isin(train_id.tweet_id)]
tweets._index = [tweets._source[i]['tweet']['tweet_id'] for i in range(0, tweets.shape[0])]
tweets['hashtags'] = [tweets._source[i]['tweet']['hashtags'] for i in range(0, tweets.shape[0])]
tweets['text'] = [tweets._source[i]['tweet']['text'] for i in range(0, tweets.shape[0])]

In [17]:
tweets['year'] = tweets._crawldate.apply(lambda x: x.split()[0].split('-')[0])
tweets['month'] = tweets._crawldate.apply(lambda x: x.split()[0].split('-')[1])
tweets['day'] = tweets._crawldate.apply(lambda x: x.split()[0].split('-')[2])
tweets['hour'] = tweets._crawldate.apply(lambda x: x.split()[1].split(':')[0])

In [8]:
tweets.drop(['_source', '_type', '_crawldate'], axis=1, inplace=True)

In [21]:
#tweets.to_csv('tweets_preprocess.csv')

## Start data processing

In [22]:
#tweets = pd.read_csv('tweets_preprocess.csv')

/tmp/ipykernel_27862/255185231.py:1: DtypeWarning: Columns (0,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv('tweets_preprocess.csv')


In [ ]:
tweets

In [ ]:
import os
import re
from itertools import chain

TAG_RE = re.compile(r'<[^>]+>')
def preprocess_text(sen):
    # Removing html tags
    sentence = TAG_RE.sub('', sen)
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

def readtweets(train_df):
    classes = ['anger', 'anticipation', 'disgust', 'fear', 'sadness', 'surprise', 'trust', 'joy']
    data = []

def label_encode(train_df): 

    label_encoder = LabelEncoder()
    label_encoder.fit(train_df.emo)  # learn categorical labels

    print('check label: ', label_encoder.classes_)
    print('\n## Before convert')
    print('train_df[0:4]:\n', train_df.emo[0:4])
    print('\ntrain_df.shape: ', train_df.shape)

    def label_encode(le, labels):
        enc = le.transform(labels)
        return keras.utils.to_categorical(enc)

    def label_decode(le, one_hot_label):
        dec = np.argmax(one_hot_label, axis=1)
        return le.inverse_transform(dec)

    train_df = label_encode(label_encoder, train_df.emo)


    print('\n\n## After convert')
    print('train_df[0:4]:\n', train_df[0:4])
    print('\ntrain_df.shape: ', train_df.shape)

def tokenizer(text):
    return [tok.lower() for tok in text.split(' ')] #簡單使用空格來斷詞

train_data = readIMDB('aclImdb','train')
test_data = readIMDB('aclImdb', 'test')

train_tokenized = []
test_tokenized = []
for review, score in train_data:
    train_tokenized.append(tokenizer(review))
for review, score in test_data:
    test_tokenized.append(tokenizer(review))

vocab = set(chain(*train_tokenized)) #把tokenized 所有字給串起來
vocab_size = len(vocab)